Code based on
    analyse_SR_results.ipynb
by 
    Fabrizio Frasca
in commit 
    bc07009fc949a32313991090a46e0f932689d7bc
which was commited
    May 15th 2021
in repository  
    twitter-research/cwn 

In [86]:
import sys
sys.path.append('./../')
import os
from definitions import ROOT_DIR
import matplotlib.pyplot as plt
import glob
import numpy as np

In [114]:
def get_results(result_path, args, which_epoch='best'):
    def parse_results(path, args):
        with open(path, 'r') as handle:
            best_val_epoch_val = None
            best_val_epoch_test = None
            final_epoch_val = None
            final_epoch_test = None

            args_dict = None

            first_valid = True
            first_test = True
            for line in handle:
                if line.startswith('Valid'):
                    # if first_valid, we view the first line that starts with 'Valid',
                    # as that is the line that shows the val acc at the best epoch.
                    # else, we view the second line that starts with 'Valid', as that
                    # is the line that shows the val acc at the last epoch.
                    if first_valid:
                        best_val_epoch_val = line.strip().split(':                  ')[1]
                        best_val_epoch_val = float(best_val_epoch_val.split(' ± ')[0])
                        first_valid = False
                    else:
                        final_epoch_val = line.strip().split(':                  ')[1]
                        final_epoch_val = float(final_epoch_val.split(' ± ')[0])
                if line.startswith('Test:'):
                    if first_test:
                        best_val_epoch_test = line.strip().split(':                   ')[1]
                        best_val_epoch_test = float(best_val_epoch_test.split(' ± ')[0])
                        first_test = False
                    else:
                        final_epoch_test = line.strip().split(':                   ')[1]
                        final_epoch_test = float(final_epoch_test.split(' ± ')[0])
                elif line.startswith('Namespace'):
                    args_dict = {}
                    fields = line.lstrip('Namespace(').rstrip(')').strip().split(', ')
                    for field in fields:
                        try:
                            key, value = field.split('=')
                            if key in args:
                                args_dict[key] = value
                        except ValueError:
                            pass
                else:
                    pass
        return best_val_epoch_val, final_epoch_val, best_val_epoch_test, final_epoch_test, args_dict

    results = list()
    for i in range(300):
        try:
            path = os.path.join(result_path, f'MOLHIV-{i}', 'result.txt')
            bval_val, fepo_val, bval_test, fepo_test, hypparam_settings = parse_results(path, args)
            if which_epoch=='best_val':
                results.append((i, bval_val, hypparam_settings))
            elif which_epoch=='final_val':
                results.append((i, fepo_val, hypparam_settings))
            elif which_epoch=='best_val_test':
                results.append((i, bval_test, hypparam_settings))
            elif which_epoch=='final_test':
                results.append((i, fepo_test, hypparam_settings))
        except FileNotFoundError:
            pass

    results.sort(reverse=True, key=lambda r:r[1])

    return results

In [117]:
result_path = os.path.join(ROOT_DIR, 'exp', 'results', 'MOLHIV_tuning_molhiv_less_sparse2')

args = [
        'lr', 
        'omit_2cell_down',
        'num_layers',
        ]

get_results(result_path, args, which_epoch='best_val')

[(34,
  0.8499779541446207,
  {'lr': '0.0003', 'num_layers': '6', 'omit_2cell_down': "'True'"}),
 (26,
  0.8478675227807172,
  {'lr': '0.0001', 'num_layers': '3', 'omit_2cell_down': "'False'"}),
 (27,
  0.8462949694624077,
  {'lr': '0.0001', 'num_layers': '4', 'omit_2cell_down': "'False'"}),
 (29,
  0.8462214832778104,
  {'lr': '0.0001', 'num_layers': '6', 'omit_2cell_down': "'False'"}),
 (36,
  0.8461536106212031,
  {'lr': '0.0003', 'num_layers': '3', 'omit_2cell_down': "'False'"}),
 (22,
  0.8452605493500557,
  {'lr': '0.0001', 'num_layers': '4', 'omit_2cell_down': "'True'"}),
 (39,
  0.8449354750473578,
  {'lr': '0.0003', 'num_layers': '6', 'omit_2cell_down': "'False'"}),
 (33,
  0.8449222067084721,
  {'lr': '0.0003', 'num_layers': '5', 'omit_2cell_down': "'True'"}),
 (38,
  0.8444833308837939,
  {'lr': '0.0003', 'num_layers': '5', 'omit_2cell_down': "'False'"}),
 (32,
  0.8438143004115227,
  {'lr': '0.0003', 'num_layers': '4', 'omit_2cell_down': "'True'"}),
 (37,
  0.84351116990005

In [111]:

result_path1 = os.path.join(ROOT_DIR, 'exp', 'results', 'MOLHIV_tuning_molhiv_less_sparse3')
result_path2 = os.path.join(ROOT_DIR, 'exp', 'results', 'MOLHIV_tuning_molhiv_less_sparse4')

args = [ 
        'lr', 
        'omit_2cell_down',
        'num_layers',
        'use_boundaries'
        ]

get_results(result_path1, args, which_epoch='final_test') + \
    get_results(result_path2, args, which_epoch='final_test')

[(0,
  0.7868308902579553,
  {'lr': '3e-05',
   'num_layers': '4',
   'omit_2cell_down': "'False'",
   'use_boundaries': "'True'"}),
 (1,
  0.7841814892781501,
  {'lr': '3e-05',
   'num_layers': '5',
   'omit_2cell_down': "'False'",
   'use_boundaries': "'True'"}),
 (0,
  0.7934798534798536,
  {'lr': '3e-05',
   'num_layers': '4',
   'omit_2cell_down': "'False'",
   'use_boundaries': "'False'"}),
 (1,
  0.7900213085098852,
  {'lr': '3e-05',
   'num_layers': '5',
   'omit_2cell_down': "'False'",
   'use_boundaries': "'False'"})]

In [119]:
result_path = os.path.join(ROOT_DIR, 'exp', 'results', 'MOLHIV_tuning_molhiv_dense')

args = [
        'lr', 
        'omit_2cell_down',
        'num_layers',
        ]

get_results(result_path, args, which_epoch='best_val')

[(36,
  0.847407978966621,
  {'lr': '0.0003', 'num_layers': '3', 'omit_2cell_down': "'False'"}),
 (28,
  0.8469798198771964,
  {'lr': '0.0001', 'num_layers': '5', 'omit_2cell_down': "'False'"}),
 (38,
  0.8454473267359068,
  {'lr': '0.0003', 'num_layers': '5', 'omit_2cell_down': "'False'"}),
 (30,
  0.8430051868998628,
  {'lr': '0.0003', 'num_layers': '2', 'omit_2cell_down': "'True'"}),
 (39,
  0.8421618818995361,
  {'lr': '0.0003', 'num_layers': '6', 'omit_2cell_down': "'False'"}),
 (37,
  0.8405630470638186,
  {'lr': '0.0003', 'num_layers': '4', 'omit_2cell_down': "'False'"}),
 (34,
  0.8402782880985041,
  {'lr': '0.0003', 'num_layers': '6', 'omit_2cell_down': "'True'"}),
 (16,
  0.8395847622313671,
  {'lr': '3e-05', 'num_layers': '3', 'omit_2cell_down': "'False'"}),
 (17,
  0.8394393208243517,
  {'lr': '3e-05', 'num_layers': '4', 'omit_2cell_down': "'False'"}),
 (31,
  0.8390974059376837,
  {'lr': '0.0003', 'num_layers': '3', 'omit_2cell_down': "'True'"}),
 (26,
  0.8389759496048076